Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Real-time Forecasting Webservice Deployment
---

In this notebook we deploy a webservice to forecast sales in real-time with the models we trained in the last step. #TODO blabla

### Prerequisites
At this point, you should have already:

1. Created your AML Workspace
2. Run 00_Environment_Setup.ipynb to configure the enviroment
3. Run 01_Training_Pipeline.ipynb to train the models

## 1.0 Connect to workspace

In [3]:
from azureml.core import Workspace

ws = Workspace.from_config()

print('Workspace Name: ' + ws.name, 
      'Azure Region: ' + ws.location, 
      'Subscription Id: ' + ws.subscription_id, 
      'Resource Group: ' + ws.resource_group, sep='\n')

Workspace Name: manymodelspip0aml
Azure Region: westeurope
Subscription Id: ed835daf-8df9-4922-a58f-74a80da0b5c0
Resource Group: manymodels-pipeline


## 2.0 Get models to be deployed

### 2.1 Get all models registered in the workspace

In [2]:
from azureml.core import Model

models = Model.list(ws, latest=True)

### 2.2 Group models by store

In [7]:
models_groups = {}
for m in models:
    
    if m.tags['ModelType'] == '_deployment_':
        continue
        
    models_store = models_groups.setdefault(m.tags['Store'], [])
    models_store.append(m)

## 3.0 Configure deployment

### 3.1 Define inference environment

In [63]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

forecast_env = Environment(name="many_models_environment")
forecast_conda_deps = CondaDependencies.create(pip_packages=['azureml-defaults', 'sklearn'])
forecast_env.python.conda_dependencies = forecast_conda_deps

### 3.2 Define inference configuration

In [64]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(
    entry_script='forecast_webservice.py',
    source_directory='./scripts',
    environment=forecast_env
)

### 3.3 Define deploy configuration

We will deploy the models to Azure Container Instances, indicated for dev/test environments.

In [9]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

## 4.0 Deploy the models

We will deploy one webservice for each of the groups of models

In [91]:
deployments = {}
for group_name, group_models in models_groups.items():
    
    service_name = 'manymodels-{}'.format(group_name).lower()
    service = Model.deploy(
        workspace=ws,
        name=service_name,
        models=group_models,
        inference_config=inference_config,
        deployment_config=aci_config,
        overwrite=True
    )
    
    print('Deploying {}...'.format(service_name))
    service.wait_for_deployment(show_output=True)
    assert service.state == 'Healthy'
    
    # Store pairs model - service where the model is deployed
    for m in group_models:
        deployments[m.name] = service.scoring_uri


Deploying manymodels-store1005...
Running...
Succeeded
ACI service creation operation finished, operation "Succeeded"
Deploying manymodels-store1006...
Running........................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Deploying manymodels-store1002...
Running.............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Deploying manymodels-store1004...
Running.............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Deploying manymodels-store1003...
Running..........................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Deploying manymodels-store1000...
Running.............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Deploying manymodels-store1001...
Running.............................
Succeeded
ACI service creation operation finished, operation "Succeeded"


## 5.0 Group all models into a single routing endpoint

### 5.1 Register deployments as an AML model

In [33]:
import joblib

joblib.dump(deployments, 'deployments.pkl')

dep_model = Model.register(
    workspace=ws, 
    model_path ='deployments.pkl', 
    model_name='models_deployed',
    tags={'ModelType': '_deployment_'},
    description='Dictionary of the service endpoint where each model is deployed'
)

Registering model models_deployed


### 5.2 Deploy routing webservice




In [68]:
routing_env = Environment(name="many_models_routing_environment")
routing_env_deps = CondaDependencies.create(pip_packages=['azureml-defaults', 'joblib'])
routing_env.python.conda_dependencies = routing_env_deps

routing_infconfig = InferenceConfig(
    entry_script='routing_webservice.py',
    source_directory='./scripts',
    environment=routing_env
)

routing_aciconfig = AciWebservice.deploy_configuration(cpu_cores=0.1, memory_gb=0.5)

routing_service = Model.deploy(
    workspace=ws,
    name='routing-manymodels',
    models=[dep_model],
    inference_config=routing_infconfig,
    deployment_config=routing_aciconfig,
    overwrite=True
)
routing_service.wait_for_deployment(show_output=True)

assert routing_service.state == 'Healthy'

print('Routing endpoint deployed with URL: {}'.format(routing_service.scoring_uri))

Running....
Succeeded
ACI service creation operation finished, operation "Succeeded"
Routing endpoint deployed with URL: http://e5dbd7b3-1145-4eb4-9d38-109cb8bc7e94.westeurope.azurecontainer.io/score


### 5.3 Test the webservice

In [78]:
test_data = {
    "store": "Store1004", "brand": "minute.maid", 
    "forecast_horizon": 5, "model_type": "lr", "date_freq": "W-THU",
    "data": {
        "dates": ["2020-04-23", "2020-04-30", "2020-05-07"],
        "values": [11450, 12235, 14713]
    }
}

test_forecast = routing_service.run(json.dumps(test_data))
test_forecast

{'store': 'Store1004',
 'brand': 'minute.maid',
 'forecast_horizon': 5,
 'model_type': 'lr',
 'date_freq': 'W-THU',
 'forecast': {'dates': ['2020-14-05',
   '2020-21-05',
   '2020-28-05',
   '2020-04-06',
   '2020-11-06'],
  'values': [15114.744450665286,
   14847.15424242685,
   14174.95421544685,
   14034.53635228468,
   14119.002027428172]}}